# 1. trainer的完整流程

```
from transformers import Trainer, TrainerArguments

training_args = TrainingArguments(...)

trainer = Trainer(...)

trainer.train()

trainer.evaluate()

trainer.predict()
```

## 1.1 prepration

In [1]:
import os
# os.environ["WORLD_SIZE"] = "2"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import load_dataset
import torch
from torch.optim import Adam
import evaluate

checkpoint = "hfl/rbt3"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def process_function(example):
    inputs = tokenizer(example['review'], truncation=True, max_length=128)
    inputs["labels"] = example['label']
    return inputs
dataset = load_dataset('csv', data_files='data/ChnSentiCorp_htl_all.csv', split='train')
dataset = dataset.filter(lambda example: example['review'] is not None)
datasets = dataset.train_test_split(test_size=0.1)
tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets['train'].column_names)

# below are no longer needed
# collator = DataCollatorWithPadding(tokenizer=tokenizer)
# train_dataloader = DataLoader(tokenized_datasets['train'], collate_fn=collator, shuffle=True, batch_size=32)
# eval_dataloader = DataLoader(tokenized_datasets['test'], collate_fn=collator, shuffle=False, batch_size=64)    

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
# below are no longer needed
# if torch.cuda.is_available():
    # model.cuda()
# optimizer = Adam(model.parameters(), lr=2e-5)  



Map:   0%|          | 0/6988 [00:00<?, ? examples/s]

Map:   0%|          | 0/777 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 1.2 create evaluation function

In [2]:
acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def eval_metrics(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='macro')
    acc.update(f1)
    return acc

## 1.3 traininig arguments

In [3]:
# train_args = TrainingArguments(output_dir='./checkpoints', use_cpu=False, report_to='none')
train_args = TrainingArguments(num_train_epochs=3,     # 训练的epoch数
                               output_dir="./checkpoints",      # 输出文件夹
                               per_device_train_batch_size=64,  # 训练时的batch_size
                               per_device_eval_batch_size=128,  # 验证时的batch_size
                               logging_steps=50,                # log 打印的频率
                               evaluation_strategy="epoch",     # 评估策略 'epoch', 'steps' then add eval_steps = 100
                               save_strategy="epoch",           # 保存策略
                               save_total_limit=3,              # 最大保存数 (模型个数)
                               learning_rate=2e-5,              # 学习率
                               weight_decay=0.01,               # weight_decay
                               metric_for_best_model="f1",      # 设定评估指标
                               load_best_model_at_end=True,      # 训练完成后加载最优模型
                               report_to='none')    
trainer = Trainer(model=model, 
                  args=train_args, 
                  train_dataset=tokenized_datasets["train"], 
                  eval_dataset=tokenized_datasets["test"], 
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                  compute_metrics=eval_metrics)

## 1.4 rock and roll

In [4]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/yutong/anaconda3/envs/IMP/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=165, training_loss=0.33478422742901426, metrics={'train_runtime': 27.2829, 'train_samples_per_second': 768.394, 'train_steps_per_second': 6.048, 'total_flos': 351909933963264.0, 'train_loss': 0.33478422742901426, 'epoch': 3.0})

## 1.5 evaluation

In [5]:
trainer.evaluate()
trainer.evaluate(tokenized_datasets["test"])

/home/yutong/anaconda3/envs/IMP/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.30267688632011414,
 'eval_accuracy': 0.8738738738738738,
 'eval_f1': 0.8539473684210526,
 'eval_runtime': 0.4029,
 'eval_samples_per_second': 1928.501,
 'eval_steps_per_second': 9.928,
 'epoch': 3.0}

In [6]:
trainer.evaluate(tokenized_datasets["train"])

/home/yutong/anaconda3/envs/IMP/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.24743223190307617,
 'eval_accuracy': 0.8993989696622782,
 'eval_f1': 0.8846196890195286,
 'eval_runtime': 3.3825,
 'eval_samples_per_second': 2065.938,
 'eval_steps_per_second': 8.278,
 'epoch': 3.0}

## 1.6 predict

In [7]:
trainer.predict(tokenized_datasets["test"])

/home/yutong/anaconda3/envs/IMP/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


PredictionOutput(predictions=array([[-1.8978134 ,  2.778994  ],
       [-0.524505  ,  0.8458282 ],
       [ 0.58819115, -0.80928445],
       ...,
       [-1.4826006 ,  2.172026  ],
       [ 0.26391464, -0.48093486],
       [-1.7576326 ,  2.824804  ]], dtype=float32), label_ids=array([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0